In [ ]:
## Create conda environment - execute in terminal
'''
conda create -n rapids-0.17 -c rapidsai -c nvidia \
-c conda-forge -c dask -c defaults rapids-blazing=0.17 python=3.7 cudatoolkit=11.0  gcsfs=0.7.1 -y

conda activate rapids-0.17
'''

In [3]:
!git clone https://github.com/remylouisew/rapids_AIP.git

Cloning into 'rapids_AIP'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 155 (delta 39), reused 131 (delta 27), pack-reused 0
Receiving objects: 100% (155/155), 107.19 KiB | 2.19 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [1]:
import dask_cudf
import cupy as cp
import argparse
import time
import gcsfs
import os, json
import subprocess
import pandas as pd

In [2]:
# Instatiate LocalCUDACluster to assign dask processes to GPUs

from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait

cluster = LocalCUDACluster()
client = Client(cluster)


In [8]:
colnames = ['label'] + ['feature-%02d' % i for i in range(1, 29)]
train_dir='gs://rwtmp_demo_ml/nvidiadask/folder/*/*.csv' #GCS public bucket
df = dask_cudf.read_csv(train_dir, header=None, names=colnames, chunksize=None)

print("Number of partitions is", df.npartitions)

Number of partitions is 24


In [5]:
train_dir

'gs://nvidiadask/folder/**/*.csv'

In [4]:
# Read dataset into Dask DMatrix from Google Cloud Storage
col_acq_names = ['LoanID','Channel','SellerName','OrInterestRate','OrUnpaidPrinc','OrLoanTerm',
        'OrDate','FirstPayment','OrLTV','OrCLTV','NumBorrow','DTIRat','CreditScore',
        'FTHomeBuyer','LoanPurpose','PropertyType','NumUnits','OccStatus','PropertyState',
        'Zip','MortInsPerc','ProductType','CoCreditScore','MortInsType','RelMortInd']
col_per_names = ['LoanID','MonthRep','Servicer','CurrInterestRate','CAUPB','LoanAge','MonthsToMaturity',
          'AdMonthsToMaturity','MaturityDate','MSA','CLDS','ModFlag','ZeroBalCode','ZeroBalDate',
          'LastInstallDate','ForeclosureDate','DispositionDate','PPRC','AssetRecCost','MHRC',
          'ATFHP','NetSaleProceeds','CreditEnhProceeds','RPMWP','OFP','NIBUPB','PFUPB','RMWPF',
          'FPWA','ServicingIndicator']

col_acq = ['LoanID','OrDate','OrUnpaidPrinc','Channel','SellerName','PropertyType','NumUnits','PropertyState']
col_per = ['LoanID','MonthRep', 'CAUPB','CLDS','ForeclosureDate']

parse_dates_acq =['OrDate','FirstPayment']
parse_dates_per =['MonthRep','MaturityDate','ZeroBalDate','LastInstallDate','ForeclosureDate','DispositionDate']

dtype_acq={ "LoanID":"int","Channel":"str","SellerName":"str","OrInterestRate":"float","OrUnpaidPrinc":"float","OrLoanTerm":"float","OrDate":"str",
   "FirstPayment":"str","OrLTV":"float","OrCLTV":"float",  "NumBorrow":"float", "DTIRat":"float", "CreditScore":"float", "FTHomeBuyer":"str",
   "LoanPurpose":"str", "PropertyType":"str", "NumUnits":"float", "OccStatus":"str",  "PropertyState":"str",  "Zip":"int", "MortInsPerc":"float",
   "ProductType":"str", "CoCreditScore":"float", "MortInsType":"float", "RelMortInd":"str"}

dtype_per={"LoanID":"int","MonthRep":"str","Servicer":"str", "CurrInterestRate":"float", "CAUPB":"float", "LoanAge":"float","MonthsToMaturity":"float",
   "AdMonthsToMaturity":"float", "MaturityDate":"str", "MSA":"float", "CLDS":"float", "ModFlag":"str", "ZeroBalCode":"float", "ZeroBalDate":"str",
    "LastInstallDate":"str",  "ForeclosureDate":"str", "DispositionDate":"str", "PPRC":"float", "AssetRecCost":"float", "MHRC":"float", "ATFHP":"float",
    "NetSaleProceeds":"float", "CreditEnhProceeds":"float","RPMWP":"float","OFP":"float","NIBUPB":"float", "PFUPB":"float", "RMWPF":"float",
   "FPWA":"str", "ServicingIndicator":"str"
}

train_dir='gs://nvidiadask/higgs1/*.csv' #GCS public bucket
df = dask_cudf.read_csv(train_dir, header=None, names=colnames, chunksize=None)

print("Number of partitions is", df.npartitions)

RuntimeError: CUDA error at: /opt/conda/envs/rapids-0.17/include/rmm/mr/device/per_device_resource.hpp:134: cudaErrorNoDevice no CUDA-capable device is detected

In [ ]:
# Some basic functions using Dask

%%time
df["key"] = df.feature02.round()
group_means = df.groupby("key").mean().persist()
wait(group_means);

group_means.head()

#group_means.compute() will output the DMatrix as a Pandas DataFrame. A good workflow would be to summarize your data using Dask, then output to pandas for plotting or other pandas functions.


In [ ]:
# Now restart the kernal so that you can instatiate a new LocalCUDAcluster, 
# this time one that will spill to host memory when the GPU memory is exceeded

import dask_cudf
import cupy as cp
import argparse
import time
import gcsfs
import dask_cudf
import os, json
import subprocess
from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait
from dask.utils import parse_bytes

cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),)
client = Client(cluster)
client


In [ ]:
# Read in larger dataset (20GB) from GCS

colnames = ['label'] + ['feature-%02d' % i for i in range(1, 29)]
train_dir='gs://nvidiadask/higgs2/*.csv' #GCS public bucket
df = dask_cudf.read_csv(train_dir, header=None, names=colnames, chunksize=None)

print("Number of partitions is", df.npartitions)

# Run the dask functions, which will require nearly double the memory available on the GPU
%%time
df["key"] = df.feature02.round()
group_means = df.groupby("key").mean().persist()
wait(group_means);

group_means.head()

In [ ]:
##spin up notebook with rapids xgboost AND use conda install